In [18]:
import numpy as np, uproot as ur, awkward as ak, pandas as pd
import matplotlib.pyplot as plt
import os, sys
import pickle
import uproot
import fnmatch
import mplhep as hep
plt.figure()
hep.style.use("CMS")
plt.close()

def gaus(x, amp, mean, sigma):
    return amp * np.exp( -(x - mean)**2 / (2*sigma**2) ) 

def phi_reconstruct(x, y, z):
    return np.degrees(np.arctan2(y, x))
    
def theta_reconstruct(x, y, z):
    return np.degrees(np.arccos(abs(z)/np.sqrt(x**2+y**2+z**2)))

def vector_angle_reconstruct(x, y, z):
    data = np.concatenate((np.array(x)[:, np.newaxis], 
                           np.array(y)[:, np.newaxis], 
                           np.array(z)[:, np.newaxis]), 
                          axis=1)
    datamean = data.mean(axis=0)
    centered_data = data - datamean

    _, _, vv = np.linalg.svd(centered_data)
    direction_vector = vv[0]
    if direction_vector[2] > 0:
        direction_vector *= -1
        
    x_vec, y_vec, z_vec = direction_vector
    
    theta = theta_reconstruct(x_vec, y_vec, z_vec)
    phi = phi_reconstruct(x_vec, y_vec, z_vec)
    
    return theta, phi


In [35]:
#files = sorted(os.listdir(f'../muography/data/'))
target_files = fnmatch.filter(files, "four_detector_target_*_1.root")
free_files   = fnmatch.filter(files, "four_detector_free_*_1.root")
files = np.concatenate([target_files,free_files])
for j, file in enumerate(files):
    if 'merge' in file: continue
    with ur.open('~/muography/data/'+file+":events") as f:
        arrays = f.arrays(filter_name=["MuographyHits.energy", "MuographyHitsContributions.time", "MuographyHits.position.x", "MuographyHits.position.y", "MuographyHits.position.z", "MCParticles.PDG", "MCParticles.generatorStatus", "MCParticles.momentum.x", "MCParticles.momentum.y", "MCParticles.momentum.z", "MCParticles.vertex.x", "MCParticles.vertex.y", "MCParticles.vertex.z", "MCParticles.mass"])

    y,x=np.histogram(ak.flatten(arrays["MuographyHits.energy"]), bins=100, range=(0, 0.004))
    bc=(x[1:]+x[:-1])/2
    MIP=list(bc[y==max(y)])[0] 
    plt.errorbar(np.array(bc)*1000,np.array(y),yerr=np.sqrt(y))
    plt.axvline(0.2*MIP*1000,label=f'MIP = {MIP*1000:.2f} MeV')
    plt.xlabel('Cell Energy (MeV)')
    plt.legend()
    plt.close()
    
    data_energy = arrays[f'MuographyHits.energy']
    
    sigma = 0.56
    
    # flatten to numpy
    flat = ak.to_numpy(data_energy.layout.content)
    noise = np.random.normal(0, sigma, size=len(flat))*MIP
    
    # add noise
    flat_smear = np.clip(flat + noise, a_min=1e-16, a_max=None)
    
    # rebuild jagged array
    offsets = ak.to_numpy(data_energy.layout.offsets)  # convert Index64 → numpy
    lengths = offsets[1:] - offsets[:-1]
    data_energy_smear = ak.unflatten(flat_smear, lengths)
    
    data_MIP_cut = data_energy_smear > 0.2*MIP
    data_cell_cut = ak.num(arrays[f'MuographyHits.energy'], axis=1) >= 2

    data_energy = data_energy[np.array(data_cell_cut)]
    data_energy_smear = data_energy_smear[np.array(data_cell_cut)]
    data_x = arrays[f'MuographyHits.position.x'][np.array(data_cell_cut)]
    data_y = arrays[f'MuographyHits.position.y'][np.array(data_cell_cut)]
    data_z = arrays[f'MuographyHits.position.z'][np.array(data_cell_cut)]    
    reco_data_angle = np.array([vector_angle_reconstruct(np.array(xi,dtype=float), np.array(yi,dtype=float), np.array(zi,dtype=float)) for xi, yi, zi in zip(data_x,data_y,data_z)])
    data_theta = ak.Array(reco_data_angle[:,0])
    data_phi = ak.Array(reco_data_angle[:,1])
    status = arrays["MCParticles.generatorStatus"]
    mc_px = arrays["MCParticles.momentum.x"][status==1][np.array(data_cell_cut)]
    mc_py = arrays["MCParticles.momentum.y"][status==1][np.array(data_cell_cut)]
    mc_pz = arrays["MCParticles.momentum.z"][status==1][np.array(data_cell_cut)]
    mc_theta = theta_reconstruct(mc_px,mc_py,mc_pz)
    mc_phi = phi_reconstruct(mc_px,mc_py,mc_pz)
    mc_PDG = arrays["MCParticles.PDG"][status==1][np.array(data_cell_cut)]
    mc_mass = arrays["MCParticles.mass"][status==1][np.array(data_cell_cut)]
    
    data_time = arrays["MuographyHitsContributions.time"][np.array(data_cell_cut)]
    status = status[status==1][np.array(data_cell_cut)]
    num = 10000
    for i in range(int(len(mc_theta)/num)):
        print(f"Processing {file} {j}/{len(files)}: {i}/{int(len(mc_theta)/num)-1}", end='\r',flush=True)
        branches = {
            "MuographyHits.position.x": data_x[i*num:(i+1)*num],
            "MuographyHits.position.y": data_y[i*num:(i+1)*num],
            "MuographyHits.position.z": data_z[i*num:(i+1)*num],
            "MuographyHits.energy_nonsmear": data_energy[i*num:(i+1)*num],
            "MuographyHits.energy": data_energy_smear[i*num:(i+1)*num],
            "MuographyHits.time": data_time[i*num:(i+1)*num],
            "MuographyHits.theta": data_theta[i*num:(i+1)*num],
            "MuographyHits.phi": data_phi[i*num:(i+1)*num], 
            "MCParticles.theta": mc_theta[i*num:(i+1)*num],
            "MCParticles.phi": mc_phi[i*num:(i+1)*num],
            "MCParticles.generatorStatus": status[i*num:(i+1)*num],
            "MCParticles.PDG": mc_PDG[i*num:(i+1)*num],
            "MCParticles.mass": mc_mass[i*num:(i+1)*num],
            "MCParticles.momentum.x": mc_px[i*num:(i+1)*num],
            "MCParticles.momentum.y": mc_py[i*num:(i+1)*num],
            "MCParticles.momentum.z": mc_pz[i*num:(i+1)*num]
        }
        
        
        with uproot.recreate(f'training/128_channel_test/{file}_{i}.root') as fout:
            fout["events"] = branches

print('Done')

Doneessing four_detector_free_3_1.root 8/10: 1/2202133


In [30]:
path_to_result = "data/training/128_channel_test"
npz_unpacked = np.load(path_to_result+"/predictions_appended_test.npz", allow_pickle=True) 

predictions_unnormalized = npz_unpacked['outputs_scaled'].item()
targets_unnormalized = npz_unpacked['targets_scaled'].item()
predictions = npz_unpacked['outputs'].item()
targets = npz_unpacked['targets'].item()
meta = npz_unpacked['meta']
predictions_theta = np.degrees(predictions_unnormalized["theta"])


In [87]:
import os
import pickle
import awkward as ak
import uproot as ur

files = os.listdir('training/test_data1/')
merged = None  # will hold concatenated dataset

for i, file in enumerate(files):
    with ur.open(f'training/test_data1/{file}:events') as f:
        arrays = f.arrays([
            "MuographyHits.energy", 
            "MuographyHits.theta", 
            "MuographyHits.phi", 
            "MCParticles.theta", 
            "MCParticles.phi"
        ])

    print(f'Processing: {file}')

    cut = [file in i for i in meta[:, 0]]
    arrays["GNN.theta"] = predictions_theta[cut]

    branches = {
        "MuographyHits.energy": ak.sum(arrays["MuographyHits.energy"], axis=-1),
        "MuographyHits.theta": arrays["MuographyHits.theta"],
        "MuographyHits.phi": arrays["MuographyHits.phi"],
        "MCParticles.theta": ak.flatten(arrays["MCParticles.theta"]),
        "MCParticles.phi": ak.flatten(arrays["MCParticles.phi"]),
        "GNN.theta": ak.flatten(arrays["GNN.theta"]),
    }

    # Wrap dict into an Awkward record array
    batch = ak.Array(branches)

    # Concatenate across files
    merged = batch if merged is None else ak.concatenate([merged, batch], axis=0)

# Save once at the end
with open("data/training/merge.pkl", "wb") as fout:
    pickle.dump(merged, fout)


Processing: square_offset_rotated_109.root
Processing: square_offset_rotated_104.root
Processing: square_offset_rotated_94.root
Processing: square_offset_rotated_103.root
Processing: square_offset_rotated_91.root
Processing: square_offset_rotated_82.root
Processing: square_offset_rotated_101.root
Processing: square_offset_rotated_111.root
Processing: square_offset_rotated_110.root
Processing: square_offset_rotated_102.root
Processing: square_offset_rotated_107.root
Processing: square_offset_rotated_105.root
Processing: square_offset_rotated_106.root
Processing: square_offset_rotated_97.root
Processing: square_offset_rotated_88.root
Processing: square_offset_rotated_85.root
Processing: square_offset_rotated_79.root
Processing: square_offset_rotated_100.root
Processing: square_offset_rotated_99.root
Processing: square_offset_rotated_87.root
Processing: square_offset_rotated_81.root
Processing: square_offset_rotated_98.root
Processing: square_offset_rotated_83.root
Processing: square_offs